In [ ]:
# +------------------------+
# |  Input Prompt          |
# +------------------------+
#          |
#          v
# +------------------------+
# |         LLM            |
# +------------------------+
#          |
#          v
# +------------------------+
# | StructuredOutputParser |
# |  (validates JSON)      |
# +------------------------+
#          |
#          v
# +------------------------+
# | Dict: {answer, source} |
# +------------------------+

In [1]:
# STEP 1: Install LangChain, Groq integration, and Pydantic.
!pip install -q langchain langchain-groq langchain_community pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [2]:
# STEP 2: Gather your spellbooks!
from langchain_groq import ChatGroq
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)
from langchain.output_parsers.structured import StructuredOutputParser
from langchain.output_parsers import ResponseSchema
from pydantic import BaseModel, Field
from IPython.display import Markdown, display
from google.colab import userdata
import os

In [3]:
# STEP 3: Whisper your secret key to the winds.
try:
    api_key = userdata.get("GROQ_API_KEY")
except Exception:
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise ValueError("GROQ_API_KEY not found. Please set it!")

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=api_key,
    temperature=0.3,
)

In [4]:
# STEP 4: Define the fields you want the LLM to produce.
# ----------------------------------------------------------
# The StructuredOutputParser uses ResponseSchema definitions
# to generate a validated output.
# ----------------------------------------------------------

response_schemas = [
    ResponseSchema(
        name="answer",
        description="A clear explanation of the concept asked about."
    ),
    ResponseSchema(
        name="source",
        description="Where the information came from, like a reference or URL."
    )
]

# Create the parser with your schema
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get format instructions you must embed in your prompt
format_instructions = parser.get_format_instructions()

In [5]:
# STEP 5: Weave the format instructions into your system message.
# ---------------------------------------------------------------
# Use {format_instructions} as a placeholder so you can supply them safely.
# ---------------------------------------------------------------

system_msg = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. Please format your response to match these instructions:\n{format_instructions}"
)

human_msg = HumanMessagePromptTemplate.from_template(
    "{input}"
)

chat_prompt = ChatPromptTemplate.from_messages([
    system_msg,
    MessagesPlaceholder(variable_name="history"),
    human_msg
])

# Partial: provide the format instructions to fill in the placeholder
chat_prompt = chat_prompt.partial(format_instructions=format_instructions)

In [6]:
# STEP 6: Keep your conversation history.
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [7]:
# STEP 7: Chain your LLM + parser + memory.
# ---------------------------------------------------------------
# ⚡ The StructuredOutputParser ensures the output is validated
# and parsed as a dict.
# ---------------------------------------------------------------

chat_chain = chat_prompt | llm | parser

# Wrap with memory support
chatbot = RunnableWithMessageHistory(
    chat_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [8]:
# STEP 8: Test it!
session_id = "chat-session-structured-001"

user_inputs = [
    "Explain what a confusion matrix is with a source.",
    "Now explain what precision and recall mean, and give a source."
]

print(f"Starting chat loop with session ID: {session_id}")

for input_text in user_inputs:
    print(f"\nUser: {input_text}")
    response = chatbot.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}}
    )
    # The StructuredOutputParser returns a dict with validated fields
    display(Markdown(f"**Answer:** {response['answer']}\n\n**Source:** {response['source']}"))

print("\n--- Stored Chat History ---")
for message in store[session_id].messages:
    print(f"{message.type.capitalize()}: {message.content}")

Starting chat loop with session ID: chat-session-structured-001

User: Explain what a confusion matrix is with a source.


**Answer:** A confusion matrix is a table used to describe the performance of a classification model, such as a logistic regression or decision tree, on a set of test data for which the true values are known. The matrix itself is relatively simple to understand, but the related terminology can be confusing. The matrix has the following structure: True Positives (TP), False Positives (FP), True Negatives (TN), and False Negatives (FN).

**Source:** https://en.wikipedia.org/wiki/Confusion_matrix


User: Now explain what precision and recall mean, and give a source.


**Answer:** Precision and recall are two fundamental metrics used to evaluate the performance of classification models in machine learning and information retrieval. Precision is the ratio of true positives (correctly predicted instances) to the sum of true positives and false positives (incorrectly predicted instances). It measures the accuracy of the model's positive predictions. Recall, on the other hand, is the ratio of true positives to the sum of true positives and false negatives (missed instances). It measures the model's ability to detect all instances of a particular class. A balance between precision and recall is often sought, as improving one can come at the expense of the other.

**Source:** https://en.wikipedia.org/wiki/Precision_and_recall


--- Stored Chat History ---
